In [1]:
#IMPORTS
import pandas as pd

In [107]:
CO2_PATH = "../input/seoul-co2/Seoul co2.csv"
POLLUTION_PATH = "../input/air-pollution-in-seoul/AirPollutionSeoul/Measurement_summary.csv"

co2_data = pd.read_csv(CO2_PATH).drop('City', axis=1)
co2_data['Date'] = pd.date_range(start="2019-01-01",end="2019-12-31").date
co2_data['Date'] = co2_data['Date'].astype(str)
co2_data.set_index('Date', inplace=True)

pollution_data = pd.read_csv(POLLUTION_PATH).drop(['Station code', 'Address', 'Latitude', 'Longitude'], axis=1)
pollution_data = pollution_data[pollution_data['Measurement date'].str.contains('2019')].set_index("Measurement date")
pollution_data = pollution_data[pollution_data > 0]

In [3]:
nan = pollution_data.isna().sum().sum()
notnan = pollution_data.notna().sum().sum()
print(f'NAN Percentage {round(nan*100/notnan, 2)}%')

NAN Percentage 1.81%


In [110]:
pollution_data.describe()

,SO2,NO2,O3,CO,PM10,PM2.5
count,206380.000000,206368.000000,206337.000000,206527.000000,204549.000000,204511.000000
mean,0.004051,0.028064,0.024966,0.535124,46.141751,28.322379
std,0.004333,0.015935,0.019903,0.430115,79.235088,59.866435
min,0.001000,0.001000,0.001000,0.100000,1.000000,1.000000
25%,0.003000,0.016000,0.009000,0.400000,22.000000,12.000000
50%,0.004000,0.025000,0.022000,0.500000,34.000000,19.000000
75%,0.005000,0.038000,0.036000,0.600000,52.000000,31.000000
max,0.378000,0.310000,1.346000,36.800000,1985.000000,995.000000


In [64]:
dates = pd.date_range(start="2019-01-01",end="2019-12-31")
values = []
for date in dates:
    day = str(date.date())
    df_temp = pollution_data[pollution_data.index.str.contains(day)]
    value = df_temp.mean().values
    values.append([day, *value])

df = pd.DataFrame(values, columns=pollution_data.columns.insert(0, 'DATE'))
df.dropna(inplace=True)
df.set_index('DATE', inplace=True)
df.sort_index(inplace=True)
df = df.merge(co2_data, left_index=True, right_index=True)

In [120]:
df

,SO2,NO2,O3,CO,PM10,PM2.5,MtCO2
2019-01-01,0.004355,0.029252,0.016249,0.659304,37.455652,26.506087,29.7263
2019-01-02,0.004744,0.036552,0.011831,0.698495,40.897993,27.041806,33.2477
2019-01-03,0.005099,0.045253,0.006621,0.767563,42.335017,32.139966,33.8556
2019-01-04,0.006185,0.058619,0.003791,1.055927,60.891486,43.934891,33.5865
2019-01-05,0.005092,0.029015,0.016855,0.647833,64.895000,45.691667,35.1023
...,...,...,...,...,...,...,...
2019-12-27,0.003433,0.028083,0.018195,0.556500,34.030000,24.896494,38.4687
2019-12-28,0.003710,0.043712,0.005955,0.677167,42.831667,25.313333,38.4315
2019-12-29,0.003337,0.037145,0.009010,0.641333,44.428333,27.306667,38.3202
2019-12-30,0.003272,0.031195,0.012347,0.628500,38.031667,27.060000,38.2882
